In [2]:
pwd

'/home/wsuser/work'

In [3]:
!pip install Keras
!pip install tensorflow

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.2,horizontal_flip=True, vertical_flip=False)
test_datagen= ImageDataGenerator(rescale=1./255)

In [6]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='_CF_zCnU3r3ob9Ykr5pZbPThZTaWYV5O-a6L4uGXFEF4',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'imageclassification-donotdelete-pr-0b7lfassq3idmq'
object_key = 'Data.zip'

streaming_body_8 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

In [7]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_8.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [8]:
pwd

'/home/wsuser/work'

In [9]:
import os
filenames=os.listdir('/home/wsuser/work/Data/training')

In [10]:
x_train = train_datagen.flow_from_directory("/home/wsuser/work/Data/training",target_size=(100,100),class_mode='categorical',batch_size=75)
x_test = test_datagen.flow_from_directory("/home/wsuser/work/Data/testing",target_size=(100,100),class_mode='categorical',batch_size=75)

Found 13000 images belonging to 26 classes.
Found 2688 images belonging to 26 classes.


In [11]:
x_train.class_indices

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25}

In [12]:
model = Sequential()

In [13]:
model.add(Convolution2D(32, (3,3), input_shape=(100,100,3),activation = 'relu')) #Feature map

In [14]:
model.add(MaxPooling2D(pool_size = (2,2))) #Pooled matrix

In [15]:
model.add(Flatten())

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 76832)             0         
                                                                 
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.add(Dense(512,activation='relu'))
model.add(Dense(256,activation='relu'))

In [18]:
model.add(Dense(26,activation='softmax'))

In [19]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [20]:
len(x_train)

174

In [21]:
len(x_test)

36

In [22]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 17.2 MB/s eta 0:00:01


In [23]:
model.save('A2Z.h5')

In [24]:
!tar -zcvf specially-abled-model_new.tgz A2Z.h5

A2Z.h5


In [25]:
ls -1

A2Z.h5
Data/
specially-abled-model_new.tgz


In [26]:
!pip install watson-machine-learning-client --upgrade

In [27]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"3rHS2yjPVRZuSI3BERNZdd12z1795y1btiV6osUdMfdU"
}
client = APIClient(wml_credentials)

In [28]:
client = APIClient(wml_credentials)

In [29]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return(next(item for item in space['resources']if item['entity']['name']== space_name)['metadata']['id'])

In [30]:
space_uid = guid_from_space_name(client,'imageclassification')
print("space UID=" + space_uid)

space UID=4ce1d0c0-8e85-4d4c-900c-4286c5a9f2c8


In [31]:
client.set.default_space(space_uid)

'SUCCESS'

In [32]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [33]:
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
print(software_spec_uid)

acd9c798-6974-5d2f-a657-ce06e986df4d


In [34]:
model_details = client.repository.store_model(model='specially-abled-model_new.tgz',meta_props={
     client.repository.ModelMetaNames.NAME:"CNN",
     client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
     client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid}
                                             )                                          
model_id=client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [35]:
model_id

'46428fef-fc3c-4b69-8287-ad67c964c3e6'

In [36]:
client.repository.download(model_id,'my_model.tar.gz')

Successfully saved model content to file: 'my_model.tar.gz'


'/home/wsuser/work/my_model.tar.gz'